# Background            

This project research identifies the closest CoV spike protein sequence alignments of the following species: Avian, Bat, Bovine, Canine, and Human using the Needleman Wunsch algorithm and the BLOSUM 62 scoring matrix. 

The BLOSUM matrix was used to define the mismatch penalty, meaning that the mismatches have different penalties depending on which amino acids are changed. To encode the maximum alignment score, we used a `score` matrix where `score[I, j]` stores the score between the first `i` characters of the first sequence and the first `j` characters of the second sequence. By referencing the previous cells around a given position, the score for that position can be determined. We captured whether a score was due to an insertion, deletion, or mismatch, in the direction matrix. Using these two matrices, it is possible to calculate the maximum alignment score, and then reconstruct the alignment.

To extract the actual alignment, we used the directions matrix we filled in while calculating scores. Starting from the bottom-right corner, we simply followed the recorded directions, updating the finished alignments according to the directions we saw. 

This whole procedure was packaged into a function that takes in two sequences and returns the alignment score, as well as the actual alignments with insertions, deletions, and mismatches. The final alignment score is just the score in the bottom-right of the scores matrix.

Using the method described above, we are able to determine optimum alignments between two proteins. This allowed for comparison of spike proteins from several species of CoV by reporting the mismatches, matches, and percent identity. Species were also all compared with each other and the BLOSUM score for each pair was reported.


In [ ]:
import numpy as np
from numba import jit, njit # Accelerates python functions

In [ ]:
# Function that adds colors to aligned sequences
from termcolor import colored 

def print_aligned_seqs(a, b):
  out1 = []
  out2 = []
  for (c, d) in zip(a, b):
    color1 = color2 = ''
    if d == '-':
      color1 = 'red'
    elif c == '-':
      color2 = 'green'
    elif c != d:
      color1 = color2 = 'blue'

    if color1 != '':
      out1.append(colored(c, color1))
    else:
      out1.append(c)

    if color2 != '':
      out2.append(colored(d, color2))
    else:
      out2.append(d)
  print("".join(out1))
  print("".join(out2))

In [ ]:
blosum_txt = (
"""A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4 
R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4 
N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4 
D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4 
C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4 
Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4 
E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4 
H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4 
I -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4 
L -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4 
K -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4 
M -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4 
F -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -3 -3 -1 -4 
P -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2 -1 -2 -4 
S  1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0  0  0 -4 
T  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0 -1 -1  0 -4 
W -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3 -4 -3 -2 -4 
Y -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1 -3 -2 -1 -4 
V  0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4 -3 -2 -1 -4 
B -2 -1  3  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3  4  1 -1 -4 
Z -1  0  0  1 -3  3  4 -2  0 -3 -3  1 -1 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -4 
* -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  1"""
)
amino_acid_order = "ARNDCQEGHILKMFPSTWYVBZX*"
num_amino_acids = len(amino_acid_order)
blosum = {}

# Split into lines and drop the first line (because we don't need it).
lines = blosum_txt.split("\n")[1:]

# Iterate over the lines
for line in lines:
  first_am = line[0]

  # Extract numbers from this line. Numbers are everything after character 2.
  numbers_txt = line[2:]

  # Split the numbers on spaces to isolate each number.
  numbers_list = numbers_txt.split(" ")

  # Filter out any empty strings (caused by 2 spaces in a row).
  numbers = [x for x in numbers_list if x != '']

  # Zip with the amino acid order we got earlier.
  # This tags each data point with its second amino acid.
  for (second_am, score_txt) in zip(amino_acid_order, numbers):

    # Convert string to int, and assign the amino acid pair this score
    pair = (first_am, second_am)
    score = int(score_txt)
    blosum[pair] = score

blosum

{('*', '*'): 1,
 ('*', 'A'): -4,
 ('*', 'B'): -4,
 ('*', 'C'): -4,
 ('*', 'D'): -4,
 ('*', 'E'): -4,
 ('*', 'F'): -4,
 ('*', 'G'): -4,
 ('*', 'H'): -4,
 ('*', 'I'): -4,
 ('*', 'K'): -4,
 ('*', 'L'): -4,
 ('*', 'M'): -4,
 ('*', 'N'): -4,
 ('*', 'P'): -4,
 ('*', 'Q'): -4,
 ('*', 'R'): -4,
 ('*', 'S'): -4,
 ('*', 'T'): -4,
 ('*', 'V'): -4,
 ('*', 'W'): -4,
 ('*', 'X'): -4,
 ('*', 'Y'): -4,
 ('*', 'Z'): -4,
 ('A', '*'): -4,
 ('A', 'A'): 4,
 ('A', 'B'): -2,
 ('A', 'C'): 0,
 ('A', 'D'): -2,
 ('A', 'E'): -1,
 ('A', 'F'): -2,
 ('A', 'G'): 0,
 ('A', 'H'): -2,
 ('A', 'I'): -1,
 ('A', 'K'): -1,
 ('A', 'L'): -1,
 ('A', 'M'): -1,
 ('A', 'N'): -2,
 ('A', 'P'): -1,
 ('A', 'Q'): -1,
 ('A', 'R'): -1,
 ('A', 'S'): 1,
 ('A', 'T'): 0,
 ('A', 'V'): 0,
 ('A', 'W'): -3,
 ('A', 'X'): 0,
 ('A', 'Y'): -2,
 ('A', 'Z'): -1,
 ('B', '*'): -4,
 ('B', 'A'): -2,
 ('B', 'B'): 4,
 ('B', 'C'): -3,
 ('B', 'D'): 4,
 ('B', 'E'): 1,
 ('B', 'F'): -3,
 ('B', 'G'): -1,
 ('B', 'H'): 0,
 ('B', 'I'): -3,
 ('B', 'K'): 0,
 ('B', 'L'

In [ ]:
# Spike glycoprotein sequences from various specimens

# SARS coronavirus Tor2, spike glycoprotein sequence (NC_004718.3)
# https://www.ncbi.nlm.nih.gov/nuccore/NC_004718.3
human_CoV_spike_seq = ("MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPD"
                     "EIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFGNPVIPFKDGIYFAATEKSNVVRGWV"
                     "FGSTMNNKSQSVIIINNSTNVVIRACNFELCDNPFFAVSKPMGTQTHTMIFDNAFNCT"
                     "FEYISDAFSLDVSEKSGNFKHLREFVFKNKDGFLYVYKGYQPIDVVRDLPSGFNTLKP"
                     "IFKLPLGINITNFRAILTAFSPAQDIWGTSAAAYFVGYLKPTTFMLKYDENGTITDAV"
                     "DCSQNPLAELKCSVKSFEIDKGIYQTSNFRVVPSGDVVRFPNITNLCPFGEVFNATKF"
                     "PSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLNDLCFSNVYADSFVVKGD"
                     "DVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYKYRYLRHGKLRP"
                     "FERDISNVPFSPDGKPCTPPALNCYWPLNDYGFYTTTGIGYQPYRVVVLSFELLNAPA"
                     "TVCGPKLSTDLIKNQCVNFNFNGLTGTGVLTPSSKRFQPFQQFGRDVSDFTDSVRDPK"
                     "TSEILDISPCAFGGVSVITPGTNASSEVAVLYQDVNCTDVSTAIHADQLTPAWRIYST"
                     "GNNVFQTQAGCLIGAEHVDTSYECDIPIGAGICASYHTVSLLRSTSQKSIVAYTMSLG"
                     "ADSSIAYSNNTIAIPTNFSISITTEVMPVSMAKTSVDCNMYICGDSTECANLLLQYGS"
                     "FCTQLNRALSGIAAEQDRNTREVFAQVKQMYKTPTLKYFGGFNFSQILPDPLKPTKRS"
                     "FIEDLLFNKVTLADAGFMKQYGECLGDINARDLICAQKFNGLTVLPPLLTDDMIAAYT"
                     "AALVSGTATAGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKQIANQFNKAIS"
                     "QIQESLTTTSTALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAE"
                     "VQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYH"
                     "LMSFPQAAPHGVVFLHVTYVPSQERNFTTAPAICHEGKAYFPREGVFVFNGTSWFITQ"
                     "RNFFSPQIITTDNTFVSGNCDVVIGIINNTVYDPLQPELDSFKEELDKYFKNHTSPDV"
                     "DLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYVWLGFIA"
                     "GLIAIVMVTILLCCMTSCCSCLKGACSCGSCCKFDEDDSEPVLKGVKLHYT"
                      )

# Avian infectious bronchitis virus, spike protein (NC_001451)
# https://www.ncbi.nlm.nih.gov/nuccore/NC_001451
avian_CoV_spike_seq = ("MLVTPLLLVTLLCALCSAVLYDSSSYVYYYQSAFRPPSGWHLQG"
                     "GAYAVVNISSEFNNAGSSSGCTVGIIHGGRVVNASSIAMTAPSSGMAWSSSQFCTAHC"
                     "NFSDTTVFVTHCYKHGGCPLTGMLQQNLIRVSAMKNGQLFYNLTVSVAKYPTFRSFQC"
                     "VNNLTSVYLNGDLVYTSNETIDVTSAGVYFKAGGPITYKVMREVKALAYFVNGTAQDV"
                     "ILCDGSPRGLLACQYNTGNFSDGFYPFTNSSLVKQKFIVYRENSVNTTCTLHNFIFHN"
                     "ETGANPNPSGVQNIQTYQTKTAQSGYYNFNFSFLSSFVYKESNFMYGSYHPSCKFRLE"
                     "TINNGLWFNSLSVSIAYGPLQGGCKQSVFKGRATCCYAYSYGGPSLCKGVYSGELDHN"
                     "FECGLLVYVTKSGGSRIQTATEPPVITQNNYNNITLNTCVDYNIYGRTGQGFITNVTD"
                     "SAVSYNYLADAGLAILDTSGSIDIFVVQGEYGLNYYKVNPCEDVNQQFVVSGGKLVGI"
                     "LTSRNETGSQLLENQFYIKITNGTRRFRRSITENVANCPYVSYGKFCIKPDGSIATIV"
                     "PKQLEQFVAPLFNVTENVLIPNSFNLTVTDEYIQTRMDKVQINCLQYVCGSSLDCRKL"
                     "FQQYGPVCDNILSVVNSVGQKEDMELLNFYSSTKPAGFNTPVLSNVSTGEFNISLLLT"
                     "NPSSRRKRSLIEDLLFTSVESVGLPTNDAYKNCTAGPLGFFKDLACAREYNGLLVLPP"
                     "IITAEMQALYTSSLVASMAFGGITAAGAIPFATQLQARINHLGITQSLLLKNQEKIAA"
                     "SFNKAIGHMQEGFRSTSLALQQIQDVVSKQSAILTETMASLNKNFGAISSVIQEIYQQ"
                     "FDAIQANAQVDRLITGRLSSLSVLASAKQAEYIRVSQQRELATQKINECVKSQSIRYS"
                     "FCGNGRHVLTIPQNAPNGIVFIHFSYTPDSFVNVTAIVGFCVKPANASQYAIVPANGR"
                     "GIFIQVNGSYYITARDMYMPRAITAGDVVTLTSCQANYVSVNKTVITTFVDNDDFDFN"
                     "DELSKWWNDTKHELPDFDKFNYTVPILDIDSEIDRIQGVIQGLNDSLIDLEKLSILKT"
                     "YIKWPWYVWLAIAFATIIFILILGWVFFMTGCCGCCCGCFGIMPLMSKCGKKSSYYTT"
                     "FDNDVVTEQYRPKKSV")

# Bat coronavirus RaTG13, spike glycoprotein sequence (MN996532.2)
# https://www.ncbi.nlm.nih.gov/nuccore/MN996532
bat_CoV_spike_seq = ("MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFR"
              "SSVLHLTQDLFLPFFSNVTWFHAIHVSGTNGIKRFDNPVLPFNDGVYFASTEKSNIIR"
              "GWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVY"
              "SSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPP"
              "GFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFL"
              "LKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTDSIVRFPNITN"
              "LCPFGEVFNATTFASVYAWNRKRISNCVADYSVLYNSTSFSTFKCYGVSPTKLNDLCF"
              "TNVYADSFVITGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSKHIDAKEGGNFN"
              "YLYRLFRKANLKPFERDISTEIYQAGSKPCNGQTGLNCYYPLYRYGFYPTDGVGHQPY"
              "RVVVLSFELLNAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFG"
              "RDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNASNQVAVLYQDVNCTEVPVAI"
              "HADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSRS"
              "VASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICG"
              "DSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNF"
              "SQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTV"
              "LPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYE"
              "NQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSV"
              "LNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVL"
              "GQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPRE"
              "GVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGSCDVVIGIVNNTVYDPLQPELDSFKE"
              "ELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQ"
              "YIKWPWYIWLGFIAGLIAIIMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLK"
              "GVKLHYT") 

# Bovine coronavirus isolate BCoV-ENT, spike structural protein (NC_003045)
# https://www.ncbi.nlm.nih.gov/nuccore/NC_003045
bovine_CoV_spike_seq = ("MFLILLISLPTAFAVIGDLKCTTVSINDVDTGVPSISTDTVDVT"
                     "NGLGTYYVLDRVYLNTTLLLNGYYPTSGSTYRNMALKGTLLLSTLWFKPPFLSDFTNG"
                     "IFAKVKNTKVIKDGVMYSEFPAITIGSTFVNTSYSVVVQPHTTILGNKLQGFLEISVC"
                     "QYTMCEYPNTICNPNLGNQRVELWHWDTGVVSCLYKRNFTYDVNADYLYFHFYQEGGT"
                     "FYAYFTDTGVVTKFLFNVYLGTVLSHYYVMPLTCNSALTLEYWVTPLTSKQYLLAFNQ"
                     "DGVIFNAVDCKSDFMSEIKCKTLSIAPSTGVYELNGYTVQPIADVYRRIPNLPDCNIE"
                     "AWLNDKSVPSPLNWERKTFSNCNFNMSSLMSFIQAYSFTCNNIDAAKIYGMCFSSITI"
                     "DKFAIPNGRKVDLQLGNLGYLQSFNYRIDTTATSCQLYYNLPAANVSVSRFNPSTWNR"
                     "RFGFTEQSVFKPQPAGVFTDHDVVYAQHCFKASTNFCPCKLDGSLCVGNGPGIDAGYK"
                     "TSGIGTCPAGTNYLTCHNAAQCDCLCTPDPITSKATGPYKCPQTKYLVGIGEHCSGLA"
                     "IKSDHCGGNPCTCQPQAFLGWSVDSCLQGDRCNIFANFILHDVNSGTTCSTDLQKSNT"
                     "DIILGVCVNYDLYGITGQGIFVEVNATYYNSWQNLLYDSNGNLYGFRDYLTNRTFMIR"
                     "SCYSGRVSAAFHANSSEPALLFRNIKCNYVFNNTLSRQLQPINYFDSYLGCVVNADNS"
                     "TSSVVQTCDLTVGSGYCVDYSTKRRSRRSITTGYRFTNFEPFTVNSVNDSLEPVGGLY"
                     "EIQIPSEFTIGNMEEFIQTSSPKVTIDCSAFVCGDYAACKSQLVEYGSFCDNINAILT"
                     "EVNELLDTTQLQVANSLMNGVTLSTKLKDGVNFNVDDINFSPVLGCLGSDCNKVSSRS"
                     "AIEDLLFSKVKLSDVGFVEAYNNCTGGAEIRDLICVQSYNGIKVLPPLLSENQISGYT"
                     "LAATSASLFPPWSAAAGVPFYLNVQYRINGIGVTMDVLSQNQKLIANAFNNALGAIQE"
                     "GFDATNSALVKIQAVVNANAEALNNLLQQLSNRFGAISSSLQEILSRLDALEAQAQID"
                     "RLINGRLTALNAYVSQQLSDSTLVKFSAAQAMEKVNECVKSQSSRINFCGNGNHIISL"
                     "VQNAPYGLYFIHFSYVPTKYVTAKVSPGLCIAGDRGIAPKSGYFVNVNNTWMFTGSGY"
                     "YYPEPITGNNVVVMSTCAVNYTKAPDVMLNISTPNLPYFKEELDQWFKNQTSVAPDLS"
                     "LDYINVTFLDLQDEMNRLQEAIKVLNQSYINLKDIGTYEYYVKWPWYVWLLIGFAGVA"
                     "MLVLLFFICCCTGCGTSCFKKCGGCCDDYTGHQELVIKTSHED")

# Canine respiratory coronavirus strain BJ232, spike protein (KX432213)
# https://www.ncbi.nlm.nih.gov/nuccore/KX432213
canine_CoV_spike_seq = ("MFLILLISLPMAFAVIGDLKCTTASINNVDTGAPSISTDVVDVT"
                     "NGLGTYYVLDRVYLNTTLLLNGYYPTSGSTYRNMALKGTLLLSTLWFKPPFLSDFIDG"
                     "VFAKVKNTKVIKDGVVYSEFPAITIGSTFVNTSYSVVVQPHTTNLDNKLQGLLEISVC"
                     "QYTMCDYPHTMCHPNLGNKRIELWHWDTGVVSCLYKRNFTYDVNADYLYFHFYQEGGT"
                     "FYAYFTDTGVVTKFLFHVYLGTVLSHYYVMPLTCNSVMTLEYWVTPLTFKQYLLAFNQ"
                     "DGVIFNAVDCKSDFMSEIKCKTLSIAPSTGVYELNGYTVQPIADVYRRIPNLPDCNIE"
                     "AWLNDKSVPSPLNWERKTFSNCNFNMSSLMSFIQADSFTCNNIDAAKIYGMCFSSITI"
                     "DKFAIPNGRKVDLQMGNLGYLQSFNYRIDTTAISCQLYYNLPAGNVSISRFNPSIWNR"
                     "RFGFTEQSVFKPQPAGVFTDHDVVYAQHCFKAPTNFCPCKLNGSLCVGSGSGIDAGYK"
                     "NSGIGTCPAGTNYLTCYNANQCDCLCTPDPILSKSTGPYKCPQTKHLVGIGEHCSGLA"
                     "IKSDYCGGNPCTCQPKAFLGWSVDSCLQGDRCNIFANFILHGVNSGTTCSTDLQKSNT"
                     "DIILGVCVNYDLYGITGQGIFVEVNATYYNSWQNLLYDSNGNLYGFRDYLTNRTFMIR"
                     "SCYSGRVSAAFHSNSSEPALLFRNIKCNYVFNNTLSRQLQPINYFDSYLGCVVNADNS"
                     "TSISVQTCDLTVGSGYCVDYSTKRRSRRSITTGYRFTNFEPFTVNSVNDSLQPVGGLY"
                     "EIQIPSEFTIGNMQEFIQTSSPKVTIDCSVFVCGDYAACKSQLVEYGSFCDNINAILT"
                     "EVNELLDTTQLQVANSLMNGVTLSTKLKDGFNFNVDDINFSPVLGCLGSECNKVSSRS"
                     "AIEDLLFSKVKLSDVGFVDAYNNCTGGAEIRDLICVQSYNGIKVLPPLLSESQISGYT"
                     "VAATFASLFPPWSAAAGVPFYLNVQYRINGIGVTMDVLSQNQKLISNAFNNALDAIQE"
                     "GFDATNSALVKIQAVVNANAEALNNLLQQLSNRFGAISASLQEILSRLDALEAQAQID"
                     "RLINGRLTALNAYVSQQLSDSTLVKFSAAQAMEKVNECVKSQSSRINFCGNGNHIISL"
                     "VQNAPYGLYFIHFSYVPTKYVTAKVSPGLCIAGDRGIAPKSGYFVNVNNTWMFTGSGY"
                     "YYPEPITGNNVVVMSTCAVNYTKAPDVMLNISTPNLPDFKEELDQWFKNQTLMAPDLS"
                     "LDYINVTFLDLQDEMNRLQEAIKVLNHSYINLKDIGTYEYYVKWPWYVWLLIGLAGVA"
                     "MLVLLFFICCCTGCGTSCFKKCGGCCDDYTGHQELVIKTLHDD")

In [ ]:
# Write a distance calculation and position finder function
def needleman_wunsch(seq1, seq2, blosum, gap_penalty):
  n = len(seq1)
  m = len(seq2)

  # Store zeroes matrix 
  score = np.zeros((m+1, n+1))

  # Create a direction matrix
  direction = np.zeros((m+1, n+1))
  left = 1
  up = 2
  diagonal = 3

  # Fill out distance matrix
  
  # First row
  for i in range(0, m+1):
    score[i, 0] = gap_penalty * i
    direction[i, 0] = left

  # First column
  for j in range(0, n+1):
    score[0, j] = gap_penalty * j
    direction[0, j] = up
  
  # Fill in all other values in the distance matrix
  for i in range(1, m+1):
    for j in range(1, n+1):
      # Calculate the score by checking the diagonal, left, and up direction

      # Check diagonal position and the score at that diagonal position
      match = score[i-1,j-1] + blosum[(seq1[j-1], seq2[i-1])]

      # Check left position and the score
      insert = score[i - 1,j] + gap_penalty
      # Check up/down position and the score 
      delete = score[i,j - 1] + gap_penalty
      
      # Record the highest score and the direction from where they came from
      score[i, j], direction[i, j] = max([(match, diagonal), (delete, up), (insert, left)])
  
  # Store alignment
  align1 = []
  align2 = []

  # Create a back-tracing while loop to check where the max score was calculated from,
  # then update the alignment to represent that position
  i = m
  j = n
  while (i>0) or (j>0):
    if direction[i,j] == left:
      align1.append('-')
      align2.append(seq2[i-1])
      # update row position
      i -= 1
    elif direction[i,j] == up:
      align1.append(seq1[j-1])
      align2.append('-')
      # update column position
      j -= 1
    elif direction[i,j] == diagonal:
      align1.append(seq1[j-1])
      align2.append(seq2[i-1])
      # update row and column position
      i -= 1
      j -= 1

  # Returns score at the bottom right
  # Since we traversed the matrix from the bottom right, the two sequences will be reversed,
  # we will need to reverse them again
  return score[m,n], "".join(align1[::-1]), "".join(align2[::-1])


In [ ]:
# Test the function with amino acids 
import time
L = 1500
#seq1 = bat_CoV_spike_seq[:L]
#seq2 = human_CoV_spike_seq[:L]

def run_alignment(seq1, seq2):
  print("Sequence 1:", seq1)
  print("Sequence 2:", seq2)

  start = time.time()
  score, align1, align2 = needleman_wunsch(seq1, seq2, blosum, gap_penalty=-10)
  end = time.time()
  print("Execution time: {:.4f} seconds".format(end - start))
  print("Score:", score)
  print_aligned_seqs(align1, align2)

  # Find the number of mismatches and matches
  num_mismatches = sum(1 for (c,d) in zip(align1, align2) if c != d)
  num_matches =    sum(1 for (c,d) in zip(align1, align2) if c == d)

  print("Number of Mismatches: ",num_mismatches)
  print("Number of Matches: ", num_matches) 
  print("Percent Identity: ", num_matches / (num_matches + num_mismatches))
  print("")

# Compare Human with Avian
print("Now comparing Human CoV alignment with Avian CoV alignment: ")
run_alignment(human_CoV_spike_seq[:L], avian_CoV_spike_seq[:L])

# Compare Human with Bat
print("Now comparing Human CoV alignment with Bat CoV alignment: ")
run_alignment(human_CoV_spike_seq[:L], bat_CoV_spike_seq[:L])

# Compare Human with Bovine
print("Now comparing Human CoV alignment with Bovine CoV alignment: ")
run_alignment(human_CoV_spike_seq[:L], bovine_CoV_spike_seq[:L])

# Compare Human with Canine
print("Now comparing Human CoV alignment with Canine CoV alignment: ")
run_alignment(human_CoV_spike_seq[:L], canine_CoV_spike_seq[:L])

Now comparing Human CoV alignment with Avian CoV alignment: 
Sequence 1: MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFGNPVIPFKDGIYFAATEKSNVVRGWVFGSTMNNKSQSVIIINNSTNVVIRACNFELCDNPFFAVSKPMGTQTHTMIFDNAFNCTFEYISDAFSLDVSEKSGNFKHLREFVFKNKDGFLYVYKGYQPIDVVRDLPSGFNTLKPIFKLPLGINITNFRAILTAFSPAQDIWGTSAAAYFVGYLKPTTFMLKYDENGTITDAVDCSQNPLAELKCSVKSFEIDKGIYQTSNFRVVPSGDVVRFPNITNLCPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLNDLCFSNVYADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYKYRYLRHGKLRPFERDISNVPFSPDGKPCTPPALNCYWPLNDYGFYTTTGIGYQPYRVVVLSFELLNAPATVCGPKLSTDLIKNQCVNFNFNGLTGTGVLTPSSKRFQPFQQFGRDVSDFTDSVRDPKTSEILDISPCAFGGVSVITPGTNASSEVAVLYQDVNCTDVSTAIHADQLTPAWRIYSTGNNVFQTQAGCLIGAEHVDTSYECDIPIGAGICASYHTVSLLRSTSQKSIVAYTMSLGADSSIAYSNNTIAIPTNFSISITTEVMPVSMAKTSVDCNMYICGDSTECANLLLQYGSFCTQLNRALSGIAAEQDRNTREVFAQVKQMYKTPTLKYFGGFNFSQILPDPLKPTKRSFIEDLLFNKVTLADAGFMKQYGECLGDINARDLICAQKFNGLTVLPPLLTDDMIAAYTAALVSGTATAGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKQIANQFNKAISQIQESLTTTSTAL

In [ ]:
# Compare all to all in a dictionary
sequences = {
    "avian": avian_CoV_spike_seq, 
    "bat": bat_CoV_spike_seq,
    "bovine": bovine_CoV_spike_seq,
    "canine": canine_CoV_spike_seq,
    "human": human_CoV_spike_seq
}
results = {}
for name1, seq1 in sequences.items():
  for name2, seq2 in sequences.items():
    res = needleman_wunsch(seq1, seq2, blosum, gap_penalty=-10)
    results[(name1, name2)] = res 

results


    

{('avian', 'avian'): (6102.0,
  'MLVTPLLLVTLLCALCSAVLYDSSSYVYYYQSAFRPPSGWHLQGGAYAVVNISSEFNNAGSSSGCTVGIIHGGRVVNASSIAMTAPSSGMAWSSSQFCTAHCNFSDTTVFVTHCYKHGGCPLTGMLQQNLIRVSAMKNGQLFYNLTVSVAKYPTFRSFQCVNNLTSVYLNGDLVYTSNETIDVTSAGVYFKAGGPITYKVMREVKALAYFVNGTAQDVILCDGSPRGLLACQYNTGNFSDGFYPFTNSSLVKQKFIVYRENSVNTTCTLHNFIFHNETGANPNPSGVQNIQTYQTKTAQSGYYNFNFSFLSSFVYKESNFMYGSYHPSCKFRLETINNGLWFNSLSVSIAYGPLQGGCKQSVFKGRATCCYAYSYGGPSLCKGVYSGELDHNFECGLLVYVTKSGGSRIQTATEPPVITQNNYNNITLNTCVDYNIYGRTGQGFITNVTDSAVSYNYLADAGLAILDTSGSIDIFVVQGEYGLNYYKVNPCEDVNQQFVVSGGKLVGILTSRNETGSQLLENQFYIKITNGTRRFRRSITENVANCPYVSYGKFCIKPDGSIATIVPKQLEQFVAPLFNVTENVLIPNSFNLTVTDEYIQTRMDKVQINCLQYVCGSSLDCRKLFQQYGPVCDNILSVVNSVGQKEDMELLNFYSSTKPAGFNTPVLSNVSTGEFNISLLLTNPSSRRKRSLIEDLLFTSVESVGLPTNDAYKNCTAGPLGFFKDLACAREYNGLLVLPPIITAEMQALYTSSLVASMAFGGITAAGAIPFATQLQARINHLGITQSLLLKNQEKIAASFNKAIGHMQEGFRSTSLALQQIQDVVSKQSAILTETMASLNKNFGAISSVIQEIYQQFDAIQANAQVDRLITGRLSSLSVLASAKQAEYIRVSQQRELATQKINECVKSQSIRYSFCGNGRHVLTIPQNAPNGIVFIHFSYTPDSFVNVTAIVGFCVKPANASQYAIV